# Chapter 3.1: RAG Fundamentals

> **Your mission**: Build "Ask HR" - an intelligent assistant that helps Contoso employees with health plan questions. But first, you need to understand what's happening under the hood.

In this notebook, we'll build RAG (Retrieval-Augmented Generation) the hard way first. Why? Because understanding the mechanics will help you debug issues and make better design decisions later.

**What you'll build:**
1. A search index with your HR documents
2. A manual RAG implementation (to feel the pain)
3. A function tool that gives your agent search capabilities

---

## Prerequisites

Complete the setup guide (`03.0-setup-guide.md`) first. You need an Azure AI Search service and your `.env` file configured.

In [ ]:
# Environment Setup & Validation
import sys
sys.path.insert(0, '..')

from workshop_utils import validate_env
validate_env()

---

## Part 1: Building the Knowledge Base

Before Ask HR can answer questions, it needs documents to search. We'll create an Azure AI Search index programmatically - this approach is reproducible, version-controlled, and CI/CD friendly.

### Step 1.1: Load Configuration

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

# Azure AI Search configuration
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
search_key = os.getenv("SEARCH_API_KEY")
index_name = os.getenv("INDEX_NAME")

# Azure OpenAI configuration
openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
chat_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")

print(f"Search Endpoint: {search_endpoint}")
print(f"Index Name: {index_name}")
print(f"OpenAI Endpoint: {openai_endpoint}")

### Step 1.2: Define the Index Schema

Run this code to define the schema, then we'll explain what each piece does:

In [ ]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    VectorSearch,
    HnswAlgorithmConfiguration,
    HnswParameters,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticSearch,
    SemanticPrioritizedFields,
    SemanticField,
)
from azure.core.credentials import AzureKeyCredential

# Define the index fields (matching Azure AI Search portal wizard schema)
# Field names: chunk_id, parent_id, chunk, title, text_vector
fields = [
    SearchField(
        name="chunk_id",
        type=SearchFieldDataType.String,
        key=True,
        sortable=True,
        filterable=True,
    ),
    SearchField(
        name="parent_id",
        type=SearchFieldDataType.String,
        filterable=True,  # For filtering chunks by source document
    ),
    SearchField(
        name="chunk",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
    SearchField(
        name="title",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
    SearchField(
        name="text_vector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=3072,  # text-embedding-3-large dimension
        vector_search_profile_name=f"{index_name}-vector-profile",
    ),
]

# Configure vector search (matching portal wizard settings)
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name=f"{index_name}-algorithm",
            parameters=HnswParameters(
                metric="cosine",
                m=4,
                ef_construction=400,
                ef_search=500,
            ),
        ),
    ],
    profiles=[
        VectorSearchProfile(
            name=f"{index_name}-vector-profile",
            algorithm_configuration_name=f"{index_name}-algorithm",
        ),
    ],
)

# Configure semantic search
semantic_config = SemanticConfiguration(
    name=f"{index_name}-semantic-configuration",
    prioritized_fields=SemanticPrioritizedFields(
        content_fields=[SemanticField(field_name="chunk")],
        title_field=SemanticField(field_name="title"),
    ),
)

semantic_search = SemanticSearch(
    default_configuration_name=f"{index_name}-semantic-configuration",
    configurations=[semantic_config],
)

# Create the index definition
index = SearchIndex(
    name=index_name,
    fields=fields,
    vector_search=vector_search,
    semantic_search=semantic_search,
)

print(f"Index schema defined with {len(fields)} fields")
print(f"Fields: {[f.name for f in fields]}")

**What you just defined:**
- **Fields** define your data structure (like database columns) - we have `chunk_id`, `chunk` (the text), `title`, and `text_vector` (embeddings)
- **Vector search config** controls how embeddings are compared (we use cosine similarity with HNSW algorithm)
- **Semantic config** enables AI-powered ranking that understands meaning, not just keywords

### Step 1.3: Create the Index

In [ ]:
# Create the index client
index_client = SearchIndexClient(
    endpoint=search_endpoint,
    credential=AzureKeyCredential(search_key),
)

# Create or update the index
try:
    result = index_client.create_or_update_index(index)
    print(f"✅ Index '{result.name}' created/updated successfully")
except Exception as e:
    print(f"❌ Error creating index: {e}")

### Step 1.4: Load and Chunk Documents

We need to:
1. Read the PDF documents from `data/index1/`
2. Split them into chunks
3. Generate embeddings for each chunk
4. Upload to the index

In [ ]:
import hashlib
from pathlib import Path
import fitz


def extract_text_from_pdf(pdf_path: str) -> str:
    """Extract text content from a PDF file."""
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    doc.close()
    return text


def chunk_text(text: str, chunk_size: int = 2000, overlap: int = 500) -> list[str]:
    """
    Split text into overlapping chunks.
    
    Default settings match the Azure AI Search portal wizard:
    - maximumPageLength: 2000 characters
    - pageOverlapLength: 500 characters
    """
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        if chunk.strip():  # Only add non-empty chunks
            chunks.append(chunk)
        start = end - overlap
    return chunks

In [ ]:

# Load documents from data/index1/
data_path = Path("data/index1")
documents = []

for pdf_file in data_path.glob("*.pdf"):
    print(f"Processing: {pdf_file.name}")
    text = extract_text_from_pdf(str(pdf_file))
    chunks = chunk_text(text)
    
    # parent_id is the hash of the source document (for grouping chunks)
    parent_id = hashlib.md5(pdf_file.name.encode()).hexdigest()
    
    for i, chunk in enumerate(chunks):
        # chunk_id uniquely identifies each chunk (matches portal wizard format)
        chunk_id = hashlib.md5(f"{pdf_file.name}_{i}".encode()).hexdigest()
        documents.append({
            "chunk_id": chunk_id,
            "parent_id": parent_id,
            "chunk": chunk,
            "title": pdf_file.name,
        })

print(f"\nTotal chunks to index: {len(documents)}")

> **On Chunk Sizes:** The default chunking (2000 chars, 500 overlap) is a reasonable starting point, but it's not optimal for every use case. Smaller chunks (~500 chars) give more precise matches but may lose context. Larger chunks (~4000 chars) preserve context but may include irrelevant info. There's no universal "best" - it depends on your documents and query patterns. Many production systems use 1000-2000 chars with 10-20% overlap.

### Step 1.5: Generate Embeddings

We'll use Azure OpenAI to generate vector embeddings for each chunk.

In [ ]:
from openai import AzureOpenAI

# Setup Azure OpenAI client for embeddings (using API key like other notebooks)
openai_client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=openai_endpoint,
    api_key=openai_api_key,
)


def get_embedding(text: str) -> list[float]:
    """Generate embedding for a text chunk."""
    response = openai_client.embeddings.create(
        input=text,
        model=embedding_deployment
    )
    return response.data[0].embedding

In [ ]:
# Generate embeddings for all chunks (this may take a few minutes)
print("Generating embeddings...")
for i, doc in enumerate(documents):
    doc["text_vector"] = get_embedding(doc["chunk"])  # Field name matches portal wizard
    if (i + 1) % 10 == 0:
        print(f"  Processed {i + 1}/{len(documents)} chunks")

print(f"✅ Embeddings generated for {len(documents)} chunks")

### Step 1.6: Upload Documents to Index

In [ ]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

# Upload documents to the index
search_client = SearchClient(
    endpoint=search_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(search_key)
)

# Upload in batches (Azure Search has a limit of 1000 documents per batch)
batch_size = 100
for i in range(0, len(documents), batch_size):
    batch = documents[i:i + batch_size]
    result = search_client.upload_documents(documents=batch)
    succeeded = sum(1 for r in result if r.succeeded)
    print(f"Uploaded batch {i // batch_size + 1}: {succeeded}/{len(batch)} succeeded")

print(f"\n✅ Document upload complete!")

---

## Part 2: Understanding RAG

Now that your knowledge base is ready, let's understand the pattern we'll use to query it.

**RAG** (Retrieval-Augmented Generation) is simple: before the LLM answers, we search for relevant documents and include them in the prompt. This grounds responses in facts rather than the model's training data.

```
┌─────────────────────────────────────────────────────────────────┐
│                        RAG Architecture                         │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   User Query ──────┬──────────────────────────────────────────► │
│                    │                                            │
│                    ▼                                            │
│         ┌─────────────────────┐                                 │
│         │   Azure AI Search   │  ◄── Vectorized Documents       │
│         │   (Retrieval)       │                                 │
│         └─────────┬───────────┘                                 │
│                   │                                             │
│                   │ Retrieved Context                           │
│                   ▼                                             │
│         ┌─────────────────────┐                                 │
│         │   LLM (Generation)  │  ◄── Query + Context            │
│         └─────────┬───────────┘                                 │
│                   │                                             │
│                   ▼                                             │
│            Grounded Response                                    │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Why does this matter?** RAG solves real problems: it prevents hallucination by grounding answers in facts, keeps knowledge current (just update your index), works within context limits (retrieve only relevant chunks), and keeps proprietary data in your control.

---

## Part 3: Manual RAG

> Now let's build Ask HR the hard way. Spoiler: it's going to be tedious. That's the point - you'll appreciate the abstractions more.

We'll implement RAG step by step:
1. Search for relevant documents
2. Manually inject the results into a prompt
3. Generate a response

### Step 3.1: Perform a Raw Search

Let's see what the search returns **before** we add any LLM processing. This is the "Retrieval" part of RAG.

In [ ]:
from azure.search.documents.models import VectorizedQuery


def search_documents(query: str, top_k: int = 3) -> list[dict]:
    """Search the index using hybrid search (text + vector)."""
    # Generate embedding for the query
    query_vector = get_embedding(query)
    
    # Create vector query targeting the text_vector field (matches portal wizard)
    vector_query = VectorizedQuery(
        vector=query_vector,
        k=top_k,
        fields="text_vector"  # Field name matches portal wizard schema
    )
    
    # Perform hybrid search
    results = search_client.search(
        search_text=query,  # Text search on 'chunk' field
        vector_queries=[vector_query],
        select=["chunk_id", "parent_id", "chunk", "title"],
        top=top_k
    )
    
    return [{
        "chunk_id": r["chunk_id"],
        "title": r["title"],
        "chunk": r["chunk"],
        "score": r["@search.score"]
    } for r in results]


# Test the search
test_results = search_documents("What mental health services are covered?")
for r in test_results:
    print(f"📄 {r['title']} (score: {r['score']:.3f})")
    print(f"   {r['chunk']}")
    print()

### Step 3.2: Manual RAG - Injecting Context into the Prompt

Now let's manually construct a prompt that includes the retrieved context. This is the core RAG pattern - **retrieve, then generate**.

In [ ]:
from agent_framework.azure import AzureOpenAIChatClient

# Create chat client using API key (same pattern as other notebooks)
chat_client = AzureOpenAIChatClient(
    endpoint=openai_endpoint,
    api_key=openai_api_key,
    api_version=api_version,
    deployment_name=chat_deployment,
)


async def manual_rag(user_question: str) -> str:
    """
    Perform RAG manually:
    1. Search for relevant documents
    2. Build a prompt with the context
    3. Generate a response
    """
    # Step 1: Retrieve relevant context
    retrieved_docs = search_documents(user_question, top_k=3)
    
    # Step 2: Format context for the prompt
    context_text = "\n\n".join([
        f"[Source: {doc['title']}]\n{doc['chunk']}" 
        for doc in retrieved_docs
    ])
    
    # Step 3: Create agent with static instructions
    agent = chat_client.as_agent(
        name="manual_rag_agent",
        instructions="""You are an HR assistant for Contoso Electronics.
Answer questions based on the provided context.
If the context doesn't contain the answer, say "I don't have that information."
"""
    )
    
    # Step 4: Combine context with user question in the message
    augmented_message = f"""## Retrieved Context:
{context_text}

## Question:
{user_question}"""
    # Step 5: Generate response using the agent
    response = await agent.run(augmented_message)
    return response.text


# Test manual RAG
question = "Do I have to pay anything for my annual checkup?"
answer = await manual_rag(question)

print(f"Question: {question}\n")
print(f"Answer: {answer}")

### The Problems with Manual RAG

That worked, but notice what you had to do: write search logic, format the context, build the prompt, handle the response. Now imagine doing this for every agent, handling multi-turn conversations (where you need to track what was already retrieved), managing token limits when context gets too long, and combining multiple knowledge sources. It's a lot of boilerplate.

In Part 4, we'll improve this with function tools. In the next notebook, Context Providers will solve these problems elegantly.

### Exercise 1: Add Metadata Filtering

Our search returns results from all documents. What if you only want results from a specific PDF?

**Your task:** Extend `search_documents()` to accept an optional `source_filter` parameter that filters by document title.

**Hints:**
- Use the `filter` parameter in `search_client.search()`
- OData filter syntax: `"title eq 'Your Document Title.pdf'"`
- Make sure to handle the case where `source_filter` is `None`

In [ ]:
def search_documents_filtered(
    query: str, 
    source_filter: str = None, 
    top_k: int = 3
) -> list[dict]:
    """Search documents with optional source filtering."""
    query_vector = get_embedding(query)
    vector_query = VectorizedQuery(vector=query_vector, k=top_k, fields="text_vector")
    
    # TODO: Build filter expression if source_filter is provided
    filter_expr = None  # Replace this line
    
    results = search_client.search(
        search_text=query,
        vector_queries=[vector_query],
        filter=filter_expr,
        select=["chunk_id", "parent_id", "chunk", "title"],
        top=top_k
    )
    return [{"title": r["title"], "chunk": r["chunk"]} for r in results]


# Test without filter (should return results from any document)
print("=== Without filter ===")
results = search_documents_filtered("deductible")
for r in results:
    print(f"  - {r['title']}")

# Test with filter (should only return results from the specified document)
print("\n=== With filter (Northwind_Health_Plus only) ===")
results = search_documents_filtered(
    "deductible", 
    source_filter="Northwind_Health_Plus_Benefits_Details.pdf"
)
for r in results:
    print(f"  - {r['title']}")

When your filter is working correctly, the second test should only return results from `Northwind_Health_Plus_Benefits_Details.pdf`. This pattern is useful when users want to scope their questions to specific documents.

<details>
<summary>Click to see solution</summary>

```python
def search_documents_filtered(
    query: str, 
    source_filter: str = None, 
    top_k: int = 3
) -> list[dict]:
    """Search documents with optional source filtering."""
    query_vector = get_embedding(query)
    vector_query = VectorizedQuery(vector=query_vector, k=top_k, fields="text_vector")
    
    # Build filter if source specified
    filter_expr = f"title eq '{source_filter}'" if source_filter else None
    
    results = search_client.search(
        search_text=query,
        vector_queries=[vector_query],
        filter=filter_expr,
        select=["chunk_id", "parent_id", "chunk", "title"],
        top=top_k
    )
    return [{"title": r["title"], "chunk": r["chunk"]} for r in results]
```

</details>

---

## Part 4: Function Tools for Search

> What if the agent could decide *when* to search, rather than us searching on every query?

Function tools give agents agency. Instead of always retrieving context, the agent calls a search tool when it needs information. Use this approach when retrieval is optional (e.g., a general assistant that sometimes needs to look things up). Use Context Providers (next notebook) when every query needs grounding.

In [ ]:
from typing import Annotated
import json
from azure.core.exceptions import HttpResponseError


class AzureAISearchTool:
    """
    A function tool that wraps Azure AI Search for use by an agent.
    
    The agent decides when to call this tool based on the user's question.
    """
    
    def __init__(self, search_client: SearchClient, index_name: str):
        self.search_client = search_client
        self.index_name = index_name
    
    def search(
        self, 
        query: Annotated[str, "The search query to find relevant documents"],
        top: Annotated[int, "Number of results to return"] = 5
    ) -> str:
        """
        Search the Contoso HR knowledge base for information about health plans,
        policies, and employee benefits.
        
        Use this tool to find:
        - Health plan details (Northwind Health Plus, Northwind Standard)
        - Coverage information, deductibles, and copays
        - Eligibility requirements and enrollment procedures
        - Company HR policies and benefits
        """
        try:
            results = self.search_client.search(
                search_text=query,
                top=top,
                query_type="semantic",
                semantic_configuration_name=f"{self.index_name}-semantic-configuration",
            )
            
            docs = []
            for doc in results:
                docs.append({
                    "content": doc.get("chunk", ""),
                    "score": doc.get("@search.score", 0),
                    "source": doc.get("title", "Unknown"),
                })
            
            return json.dumps(docs, indent=2)
            
        except HttpResponseError as e:
            return json.dumps({"error": str(e)})

In [ ]:
# Create the search tool
search_tool = AzureAISearchTool(search_client=search_client, index_name=index_name)

# Create an agent with the search tool
tool_agent = chat_client.as_agent(
    name="hr_assistant",
    instructions="""
    You are an HR Assistant for Contoso Electronics specializing in employee health plans.
    
    IMPORTANT RULES:
    1. ALWAYS use the search tool before answering questions about health plans
    2. Base your responses ONLY on information from the search results
    3. If the search returns no relevant information, say so explicitly
    4. Cite which plan (Northwind Health Plus or Standard) the information comes from
    """,
    tools=[search_tool.search]
)

# Test the tool-based agent
response = await tool_agent.run("What are the out-of-network copays?")
print(response.text)

In [ ]:
# Test with a question that spans multiple topics
response = await tool_agent.run(
    "Compare the deductibles between Northwind Health Plus and Northwind Standard"
)
print(response.text)

---

### Exercise 2: Multi-Query Search for Better Recall

A single search query might miss relevant documents due to vocabulary mismatch. For example, searching "cost" might miss documents that say "price" or "fee". 

**Multi-query search** improves recall by:
1. Generating multiple query variations (synonyms, rephrasings)
2. Running each query against the index
3. Combining and deduplicating results

This is a real production technique - many RAG systems use an LLM to generate query variations before searching.

**Your task:** Implement `multi_query_search()` that takes a list of query variations and returns deduplicated results.

In [ ]:
# First, let's see the problem - different phrasings return different results
queries = [
    "What is the cost of the health plan?",
    "How much do I pay for coverage?",
    "What are the premium fees?"
]

for q in queries:
    results = search_documents(q, top_k=2)
    print(f"Query: {q}")
    for r in results:
        print(f"  - {r['chunk'][:80]}...")
    print()

In [ ]:
def multi_query_search(queries: list[str], top_k: int = 3) -> list[dict]:
    """
    Search with multiple query variations and combine results.
    
    Args:
        queries: List of query variations (e.g., synonyms, rephrasings)
        top_k: Number of results per query
    
    Returns:
        Deduplicated list of results, keeping highest score for duplicates
    """
    # TODO: 
    # 1. Run search_documents() for each query
    # 2. Combine all results
    # 3. Deduplicate by chunk content (keep the one with highest score)
    # Hint: Use a dict with chunk text as key to deduplicate
    
    all_results = []
    
    # Your implementation here
    
    return all_results


# Test: these queries should find more relevant docs together than any single query
query_variations = [
    "What is the deductible?",
    "How much do I pay before insurance kicks in?",
    "Out of pocket amount before coverage"
]

combined_results = multi_query_search(query_variations, top_k=2)
print(f"Found {len(combined_results)} unique results from {len(query_variations)} queries")
for r in combined_results[:5]:  # Show top 5
    print(f"  - [{r.get('score', 0):.3f}] {r['chunk'][:60]}...")

With multi-query search, you should get more comprehensive results than any single query alone. In production, you'd use an LLM to generate these query variations automatically based on the user's question.

<details>
<summary>Click to see solution</summary>

```python
def multi_query_search(queries: list[str], top_k: int = 3) -> list[dict]:
    """Search with multiple query variations and combine results."""
    seen = {}  # chunk -> result with highest score
    
    for query in queries:
        results = search_documents(query, top_k=top_k)
        for r in results:
            chunk = r['chunk']
            # Keep the result with the highest score
            if chunk not in seen or r.get('score', 0) > seen[chunk].get('score', 0):
                seen[chunk] = r
    
    # Return sorted by score descending
    return sorted(seen.values(), key=lambda x: x.get('score', 0), reverse=True)
```

</details>

---

**Bonus Exercise:** Wrap `multi_query_search` as a tool and create an agent that generates query variations from a single user question. The agent would take a question like "What's covered for dental?" and automatically generate variations like "dental coverage", "teeth benefits", "orthodontic services" before searching. This is how production RAG systems improve recall without requiring users to think of synonyms.

---

## Summary

You just built Ask HR the hard way - and felt the pain. You:

1. Created a search index programmatically (reproducible, version-controlled)
2. Built manual RAG and saw how much boilerplate it requires
3. Wrapped search as a function tool to give agents control over when to retrieve

**The key insight:** Manual RAG works, but it's tedious. You're responsible for search logic, prompt construction, multi-turn state, and token management - for every agent.

### Production Considerations: Blob Storage Integration

In this notebook, we indexed documents programmatically from local files. This approach is great for learning and development, but in production scenarios you'll want a more streamlined pipeline.

**The recommended production approach** is to connect Azure Blob Storage directly to Azure AI Search using indexers. This enables:
- **Automatic indexing** - Documents uploaded to Blob Storage are automatically indexed
- **Incremental updates** - Only new or changed documents are re-indexed
- **Managed identity authentication** - Secure, keyless connections between services
- **Built-in skillsets** - Azure AI Search can automatically chunk, embed, and enrich documents

For a complete guide on setting up this production-ready architecture with Blob Storage, managed identities, and automated indexing pipelines, see **[guides/blob-storage-ai-search-integration.md](../guides/blob-storage-ai-search-integration.md)**.

### Next: Context Providers

In **03.2**, you'll see how Context Providers eliminate this boilerplate. They automatically inject relevant context before every LLM call, handle multi-turn conversations, and make combining multiple knowledge sources trivial. Ask HR is about to get a lot easier to build.